# Foundations of Computational Economics #23

by Fedor Iskhakov, ANU

<img src="_static/img/dag3logo.png" style="width:256px;">

## More on Newton-Raphson method

<img src="_static/img/lecture.png" style="width:64px;">

<img src="_static/img/youtube.png" style="width:65px;">

[https://youtu.be/DKG_bJhVbsM](https://youtu.be/DKG_bJhVbsM)

Description: Failures of Newton method, domain of attraction. Multivariate Newton for optimization problems.

### Another look at Newton method

- iterative solver  
- Newton step $ x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)} $  
- fast (quadratic) convergence  


Today:
- potential problems
- multivariate version

In [ ]:
def newton(fun,grad,x0,tol=1e-6,maxiter=100,callback=None):
    '''Newton method for solving equation f(x)=0
    with given tolerance and number of iterations.
    Callback function is invoked at each iteration if given.
    '''
    for i in range(maxiter):
        x1 = x0 - fun(x0)/grad(x0)
        err = abs(x1-x0)
        if callback != None: callback(iter=i,err=err,x0=x0,x1=x1,fun=fun)
        if err<tol: break
        x0 = x1
    else:
        raise RuntimeError('Failed to converge in %d iterations'%maxiter)
    return x1

In [ ]:
f = lambda x: -4*x**3+5*x+1
g = lambda x: -12*x**2+5
x = newton(f,g,x0=-2.5,maxiter=7)
print('Solution is x=%1.3f, f(x)=%1.12f' % (x,f(x)))

In [ ]:
# make nice plot
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

def newton_pic(f,g,x0,    # function, gradient, initial point
               a=0,b=1,   # bounds for the picture
               **kwargs):  # additional parameters passed to solver
    '''Illustrate the Newton method iterations'''
    xd = np.linspace(a,b,1000)       # x grid
    plt.plot(xd,f(xd),c='red')       # plot the function
    plt.plot([a,b],[0,0],c='black')  # plot zero line
    ylim=[f(a),min(f(b),0)]
    def plot_step(**kwargs):
        plot_step.counter += 1
        x0,x1 = kwargs['x0'],kwargs['x1']
        f0 = kwargs['fun'](x0)
        plt.plot([x0,x0],[0,f0],c='green')
        plt.plot([x0,x1],[f0,0],c='green')
    plot_step.counter = 0
    try:
        newton(f,g,x0,callback=plot_step,**kwargs)
        print('Converged in %d steps'%plot_step.counter)
    except RuntimeError:  # catch the Runtime convergence error
        print('Failed to converge in %d iterations!'%plot_step.counter)
    plt.xlim((a,b))
    plt.show()

In [ ]:
newton_pic(f=f,g=g,x0=-2.5,a=-3,b=-.5)

### When does Newton fail?

1. Multiple solutions — *be aware*  
1. Failures  
  - diversion (domain of attraction)  
  - cycles  
  - function domain  
  - differentiability  
1. Reduced performance — may be slower in special cases  

#### Multiple solutions

In [ ]:
x0 = -2.5  # -2.5 -0.595 0.46
newton_pic(f=f,g=g,x0=-2.5,a=-3,b=1.5)

#### Diversion

In [ ]:
f = lambda x: np.arctan(x)
g = lambda x: 1/(1+x**2)
newton_pic(f=f,g=g,x0=1.25,a=-20,b=20)
newton_pic(f=f,g=g,x0=1.5,a=-20,b=20,maxiter=8)

#### Cycles

In [ ]:
f = lambda x: -4*x**3+5*x+1 # function
g = lambda x: -12*x**2+5    # derivative
x0 = -0.5689842546416416
newton_pic(f,g,x0,a=-1.5,b=1.5,maxiter=15)

In [ ]:
f = lambda x: -4*x**3+5*x+1 # function
g = lambda x: -12*x**2+5    # derivative
h = lambda x: -24*x         # second derivative
# find the unfortunate looping x0
ns = lambda x0: x0 - f(x0)/g(x0) # Newton step
ds = lambda x0: f(x0)*h(x0)/g(x0)**2 # derivative of ns
f2 = lambda x0: ns(ns(x0))-x0
g2 = lambda x0: ds(ns(x0))*ds(x0)-1

x0 = newton(f2,g2,x0=-0.56,tol=1e-16)  # find the cycling starting point
print('To cycle start with x0 = %1.16f'%x0)

newton_pic(f,g,x0,a=-1.5,b=1.5,maxiter=15)

#### Function domain and differentiability

- undefined value of function or the derivative during Newton steps  

In [ ]:
f = lambda x: np.log(x)
g = lambda x: 1/x
x0 = 2.9
newton_pic(f,g,x0,a=0.001,b=3)

#### Suboptimal performance

In [ ]:
f = lambda x: x**9  # VERY SPECIAL CASE
g = lambda x: 9*x**8
x0 = 1.0
newton_pic(f,g,x0,a=-1.5,b=1.5)
def print_err(**kwargs):
    print('{:4d}:  x = {:17.14f}  err = {:8.6e}'.format(kwargs['iter'],kwargs['x1'],kwargs['err']))
newton(f,g,x0,callback=print_err)

#### Conclusions for the use of Newton-Raphson method in economics

- many of the problematic cases rarely happen!  
- the problems that are real in economic applications:
  1. multiple solutions
  2. function domain constraints
  3. domain of attraction  
- good idea to combine slow robust solver (to get into the domain of attraction) with Newton method (for fast convergence)
  - successive approximations + Newton method is a classic approach in estimation of dynamic programming models (NFXP estimator, Rust 1987)  

#### Newton fractals

- visualizing the domain of attraction for particular problems  


$$
x^3 - 1 = 0
$$

- solutions $ x=1 $ and two more in the complex number plan $ x=1 $  
- draw the number of iterations to converge to any solution  

<img src="_static/img/newton_fractal.png" style="width:1200px;">

[https://www.youtube.com/watch?v=MWD2A0Vg2V0](https://www.youtube.com/watch?v=MWD2A0Vg2V0)

### Multivariate Newton-Raphson

Consider a system of $ n $ non-linear equations with $ n $ variables, given by

$$
G(x) = 0,
$$

- $ x \in \mathbb{R}^n $ and $ G(x) $ is $ n $-valued function.  
- **Jacobian matrix** is $ n $-by-$ n $ matrix of partial derivatives $ \nabla G(x) $  
- Newton step:  


$$
x_{i+1} = x_{i} - \big( \nabla G(x_i) \big)^{-1} G(x_i)
$$

#### Maximization of multivariate function

- classic example for multivariate Newton is solution of F.O.C. in multivariate optimization problems  
- consider $ F(x) \rightarrow \max $, where $ x \in \mathbb{R}^n $  
- let $ G(x) $ be the **gradient** of $ F(x) $, which is $ n $-valued function (derivatives with respect to each variable).  


$$
G(x) = \nabla F(x) = 0
$$

- Newton step in this case $ x_{i+1} = x_{i} - \big( \nabla^2 F(x_i) \big)^{-1} \nabla F(x_i) $  
- $ \nabla^2 F(x) $ is $ n $-by-$ n $ matrix of second derivatives is **Hessian matrix**  

#### Example

$$
F(x,y) = 2.575 - 2 \cos(x) \cos(y+\pi) - 0.575 \cos(1.25 \pi - 2x)
$$

In [ ]:
F = lambda x, y: 2.575 - 2*np.cos(x)*np.cos(y+np.pi) - 0.575*np.cos(1.25*np.pi - 2*x)

def contour_plot(fun,levels=20,bound=1,npoints=100,ax=None):
    '''Make a contour plot for illustrations'''
    xx = np.linspace(-bound, bound, npoints)
    yy = np.linspace(-bound, bound, npoints)
    X,Y = np.meshgrid(xx, yy)
    Z = fun(X, Y)
    if ax==None:
        fig, ax = plt.subplots(figsize=(10,8))
    cnt = ax.contour(X,Y,Z, vmin=Z.min(), vmax=Z.max(),levels=np.linspace(Z.min(),Z.max(),levels))
    ax.set_aspect('equal', 'box')
    return cnt

In [ ]:
contour_plot(F)
contour_plot(F,bound=np.pi)

#### Jacobian and Hessian

$$
F(x,y) = 2.575 - 2 \cos(x) \cos(y+\pi) - 0.575 \cos(1.25 \pi - 2x)
$$

$$
G(x,y) =
\begin{pmatrix}
2 \sin(x) \cos(y+\pi) - 1.15 \sin(1.25\pi - 2x) \\
2 \cos(x) \sin(y+\pi) \\
\end{pmatrix}
$$

$$
H(x,y) =
\begin{pmatrix}
2 \cos(x) \cos(y+\pi) + 2.3 \cos(1.25\pi - 2x) & -2 \sin(x) \sin(y+\pi) \\
-2 \sin(x) \sin(y+\pi) & 2 \cos(x) \cos(y+\pi) \\
\end{pmatrix}
$$

In [ ]:
G = lambda x, y: [2*np.sin(x)*np.cos(y+np.pi) - 2*0.575 * np.sin(1.25*np.pi - 2*x),
                  2*np.cos(x)*np.sin(y+np.pi)]
H = lambda x, y: [[2*np.cos(x)*np.cos(y+np.pi) - 2*0.575 * np.sin(1.25*np.pi - 2*x),
                   -2*np.sin(x)*np.sin(y+np.pi)],
                  [-2*np.sin(x)*np.sin(y+np.pi),
                    2*np.cos(x)*np.cos(y+np.pi)]]

In [ ]:
fig, axs = plt.subplots(1, 2)
contour_plot(lambda x,y: G(x,y)[0],ax=axs[0],bound=np.pi)
contour_plot(lambda x,y: G(x,y)[1],ax=axs[1],bound=np.pi)

In [ ]:
fig, axs = plt.subplots(2, 2)
contour_plot(lambda x,y: H(x,y)[0][0],ax=axs[0,0],bound=np.pi)
contour_plot(lambda x,y: H(x,y)[0][1],ax=axs[0,1],bound=np.pi)
contour_plot(lambda x,y: H(x,y)[1][0],ax=axs[1,0],bound=np.pi)
contour_plot(lambda x,y: H(x,y)[1][1],ax=axs[1,1],bound=np.pi)

In [ ]:
def newton2(fun,grad,x0,tol=1e-6,maxiter=100,callback=None):
    '''Newton method for solving equation f(x)=0, x is vector of 2 elements,
    with given tolerance and number of iterations.
    Callback function is invoked at each iteration if given.
    '''
    # conversion to array function of array argument
    npfun = lambda x: np.asarray(fun(x[0],x[1]))
    npgrad = lambda x: np.asarray(grad(x[0],x[1]))
    for i in range(maxiter):
        x1 = x0 - np.linalg.inv(npgrad(x0)) @ npfun(x0)  # matrix version
        err = np.amax(np.abs(x1-x0))  # vector version
        if callback != None: callback(iter=i,err=err,x0=x0,x1=x1,fun=fun)
        if err<tol: break
        x0 = x1
    else:
        raise RuntimeError('Failed to converge in %d iterations'%maxiter)
    return x1

In [ ]:
fig, ax = plt.subplots(1,1)
def plot_step(**kwargs):
    plot_step.counter += 1
    x0,x1 = kwargs['x0'],kwargs['x1']
    b = max(np.amax(np.abs(x0)),np.amax(np.abs(x1)))+1
    if plot_step.counter == 1 or b>plot_step.bound:
        plot_step.bound=b  # save the bound for later calls
        if plot_step.counter > 1:
            # remove old conrours if resdrawing
            for c in plot_step.contours.collections:
                c.remove()
        plot_step.contours = contour_plot(F,bound=b,ax=ax)
    ax.plot([x0[0],x1[0]],[x0[1],x1[1]],c='r')
    if plot_step.counter == 1:
        ax.scatter(x0[0],x0[1],c='r',edgecolors='r')
plot_step.counter = 0

In [ ]:
x0 = np.array([-1.8,-.2])
# x0 = np.array([-.8,-.2])
# x0 = np.array([-.2,1.8])
# x0 = np.array([-.2,-.2])
# x0 = np.array([-.25,1.5])
xs = newton2(G,H,x0,callback=plot_step)
print('Converged in %d steps'%plot_step.counter)
print('x*    = (%1.5f,%1.5f)'%tuple(xs))
print('G(x*) = (%1.5e,%1.5e)'%tuple(G(*tuple(xs))))

### Further learning resources

- Oscar Veliz videos on Newton method and domains of attraction
  [https://www.youtube.com/watch?v=E24zUEKqgwQ](https://www.youtube.com/watch?v=E24zUEKqgwQ)
  [https://www.youtube.com/watch?v=zyXRo8Qjj0A](https://www.youtube.com/watch?v=zyXRo8Qjj0A)
  [https://www.youtube.com/watch?v=MWD2A0Vg2V0](https://www.youtube.com/watch?v=MWD2A0Vg2V0)  
- NFXP estimator
  [https://en.wikipedia.org/wiki/Dynamic_discrete_choice#Estimation_with_nested_fixed_point_algorithm](https://en.wikipedia.org/wiki/Dynamic_discrete_choice#Estimation_with_nested_fixed_point_algorithm)  